# Lesson 1: Simple ReAct Agent from Scratch

In [1]:
# based on https://til.simonwillison.net/llms/python-react-pattern

# Setup local LLM

ChatGroq
- https://api.python.langchain.com/en/latest/chat_models/langchain_groq.chat_models.ChatGroq.html

In [39]:
import os
import re
from pprint import pprint

from langchain_groq import ChatGroq
from dotenv import load_dotenv

load_dotenv()

GROQ_LLM = ChatGroq(
    api_key = os.getenv('GROQ_API_KEY'),
    model = "llama3-70b-8192",
    temperature=0.0,
)

from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

# Test a simple message
- Model invoke
    - https://www.restack.io/docs/langchain-knowledge-langchain-invoke-integration
- Prompt template
    - https://www.mirascope.io/post/langchain-prompt-template

In [9]:
# Test "Hello world"
from langchain_core.prompts import ChatPromptTemplate

# Define roles and placeholders
chat_template = ChatPromptTemplate.from_messages(
    [
    ("system", ""),
    ("user", "{user_input}"),
    ("ai", ""),
    ]
)

prompt = chat_template.format_messages(user_input="Hello world")
result = GROQ_LLM.invoke(prompt)
print(result.content)

The classic "Hello world"!

This is a traditional greeting in the world of computer programming, often used as a test message to verify that a system or program is working correctly.

So, what brings you here today? Do you have any questions or topics you'd like to discuss? I'm all ears (or rather, all text)!


In [3]:
# Test "Hello world," again
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

    <|eot_id|><|start_header_id|>user<|end_header_id|>
    Hello world
    
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["message"],
)

MESSAGE = ''

responder = prompt | GROQ_LLM | StrOutputParser()

result = responder.invoke({"message": MESSAGE})

print(result)

It looks like you're trying to say "Hello, World!", a classic phrase often used as a first program in programming languages. Is that correct?


In [47]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = GROQ_LLM.invoke(self.messages)
        return completion.content


In [48]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [69]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

# Add more wording choices
known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight,
    "weight": average_dog_weight,
}

In [50]:
abot = Agent(prompt)

In [51]:
result = abot("How much does a toy poodle weigh?")
print(result)

Thought: I should look up the dog's weight using average_dog_weight
Action: average_dog_weight: Toy Poodle
PAUSE


In [52]:
result = average_dog_weight("Toy Poodle")

In [53]:
result

'a toy poodles average weight is 7 lbs'

In [54]:
next_prompt = "Observation: {}".format(result)

In [55]:
abot(next_prompt)

'Answer: A toy poodle weighs 7 lbs.'

In [56]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

In [58]:
abot = Agent(prompt)

In [59]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

'Thought: I need to find the weight of each dog breed and then add them together.'

In [60]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


In [61]:
abot(next_prompt)

'Thought: Now that I have the weight of the Border Collie, I need to find the weight of the Scottish Terrier.'

In [62]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


In [63]:
abot(next_prompt)

'Thought: Now that I have the weights of both breeds, I need to add them together to get their combined weight.'

In [64]:
next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)

Observation: 57


In [65]:
abot(next_prompt)

'Answer: The combined weight of the Border Collie and the Scottish Terrier is 57 lbs.'

### Add loop 

In [66]:
# python regular expression to selection action
action_re = re.compile(r'^Action: (\w+): (.*)$')   


In [71]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        print(actions)
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

# Why it stops without selecting the next action???
- Probably the response not verbose enough

In [72]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

Thought: I need to find the weight of each dog breed and then add them together.
[]
